# Fonctions utiles pour le script principal *ray_tracing.ipynb*

*Importation des modules*

In [ ]:
import vtk
import numpy
import random

*Conversion list/tuple en numpy & vice-versa*

Les deux fonctions *lamba* permettent de convertir une liste ou un tuple en un tableau numérique et vice-versa. 
En effet, VTK utilise et génére des données telles que les coordonnées ou les vecteurs en object de type 'liste' ou 'tuple' qu'il faut convertir en objet de type 'numpy' pour effectuer des opérations vectoriels.

In [ ]:
l2n = lambda l: numpy.array(l)
n2l = lambda n: list(n)

*Tracer un ligne entre deux points*

In [1]:
def addLine(renderer, p1, p2, color=[0.0, 0.0, 1.0], opacity=1.0): #color = RGB
    line = vtk.vtkLineSource()
    line.SetPoint1(p1)
    line.SetPoint2(p2)

    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputConnection(line.GetOutputPort())

    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    actor.GetProperty().SetColor(color)
    actor.GetProperty().SetOpacity(opacity)
    actor.GetProperty()

    renderer.AddActor(actor)

*Tracer un point*

In [ ]:
def addPoint(renderer, p, color=[0.0, 0.0, 0.0], radius=0.01, opacity=1):
    point = vtk.vtkSphereSource()
    point.SetCenter(p)
    point.SetRadius(radius)
    point.SetPhiResolution(35)
    point.SetThetaResolution(35)

    mapper = vtk.vtkPolyDataMapper()
    mapper.SetInputConnection(point.GetOutputPort())

    actor = vtk.vtkActor()
    actor.SetMapper(mapper)
    actor.GetProperty().SetColor(color)
    actor.GetProperty().EdgeVisibilityOn()  # show edges/wireframe
    actor.GetProperty().SetOpacity(opacity)
    actor.GetProperty().SetEdgeColor(0, 0, 0)

    renderer.AddActor(actor)
    
    return point

*Attribuation d'une couleur en fonction du spectre visible*

Pour mieux visualiser le ray-tracing, les couleurs des rayons et des lampes sont attribuées selon le spectre visible des longueurs d'onde. 

<img src ="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e3/Spectre_visible_lineaire_en_longueur_d_onde.png/1200px-Spectre_visible_lineaire_en_longueur_d_onde.png" width="500">


In [ ]:
def color(wavelenght):
    if wavelenght < 400:
        color = (0.54, 0.22, 0.69)
        
    if 400 <= wavelenght <500:
        color = (0, 0, 1)
        
    if 500 <= wavelenght <550:
        color = (0, 1, 0)
        
    if 550 <= wavelenght <600:
        color = (1, 1, 0)
        
    if 600 <= wavelenght <650:
        color = (1, 0.5, 0)
        
    if  wavelenght > 650:
        color = (1, 0, 0)
        
    return colo

*Intersection*

La fonction *IntersectWithLine* couplée avec *obbTree* permet de déterminer si une intersection d'une droite entre deux points avec un object préalablement défini comme intersective est trouvée.

In [ ]:
def isHit(obbTree, pSource, pTarget):
    r"""Returns True if the line intersects with the mesh in 'obbTree'"""
    code = obbTree.IntersectWithLine(pSource, pTarget, None, None)
    if code==0:
        return False
    return True

*Identification & coordonnées du point incident*

Cette fonction permet d'extraire les coordonnées et le numéro du point incident lorqu'une intersection est trouvée. 

In [ ]:
def GetIntersect(obbTree, pSource, pTarget):
    
    points = vtk.vtkPoints()
    cellIds = vtk.vtkIdList()

    code = obbTree.IntersectWithLine(pSource, pTarget, points, cellIds)
    

    pointData = points.GetData()
    noPoints = pointData.GetNumberOfTuples()
    noIds = cellIds.GetNumberOfIds()
    
    assert (noPoints == noIds)
    
    pointsInter = []
    cellIdsInter = []
    for idx in range(noPoints):
        pointsInter.append(pointData.GetTuple3(idx))
        cellIdsInter.append(cellIds.GetId(idx))
    
    return pointsInter, cellIdsInter

*Vecteur réféchi spéculaire*

Cette fonction permet de tracer le vecteur réfléchi de façon spéculaire selon la loi de Snell-Descartes (Greve,2004).

In [ ]:
def calcVecR(vecInc, vecNor):
    vecInc = l2n(vecInc)
    vecNor = l2n(vecNor)
    
    vecRef = vecInc - 2*numpy.dot(vecInc, vecNor)*vecNor
    
    return n2l(vecRef)

*Similarité cosinus*

Cette fonction permet de déterminer l'angle entre deux vecteurs $A$ et $B$ selon le concept de la similarité cosinus.

$$
\theta = arccos\left(\frac {AB} {\lVert A\lVert \lVert B\lVert}\right) 
$$

In [5]:
def calcAngle(A,B): 
    A = l2n(A)
    B = l2n(B)
    
    prodscal = A[0] * B[0] + A[1] * B[1] + vA[2] * vB[2]
    NormeA = numpy.sqrt(A[0]**2 + A[1]**2 + A[2]**2)
    NormeB = numpy.sqrt(B[0]**2 + B[1]**2 + B[2]**2)
   
    return numpy.arccos( prodscal / (NormeA * NormeB))

*Coefficient Fresnel & tirage aléatoire*

Les probabilités de réflection et de transmission sont calculées selon les coefficient de Fresnel (voir rappels théoriques). Un tirage aléatoire est effectué entre la réflection et la transmission selon le poids attribué pour chacun de deux évènements.

In [2]:
def Fresnel(n1, n2, vecInc, vecNor):
    vecInc = l2n(vecInc)
    vecNor = l2n(vecNor)
    
    thetai = calcAngle(vecInc,vecNor)
    
    if thetai < numpy.arcsin((n1/n2)*sin(thetai)):
      sin_thetat_2 = ((n1/n2)**2)*numpy.sin(thetai)**2
      cos_thetat = numpy.sqrt(1-sin_thetat_2)
    
      R_parallele = ((n1*numpy.cos(thetai) - n2*cos_thetat)/(n1*numpy.cos(thetai) + n2*cos_thetat))**2
      R_perpendiculaire = ((n2*numpy.cos(thetai) - n1*cos_thetat)/(n2*numpy.cos(thetai) + n1*cos_thetat))**2
    
      R = (R_parallele + R_perpendiculaire)/2
      T = 1-R #transmis
    
    else:
       R = 1
       T = 0  
        
    x = random.choices(['R', 'T'], weights = [R, T])
    
    return x

*Coordonnées et normal des points*

Les coordonnées des cellules du maillage d'une forme, ainsi que leur normale sont déterminées.

In [ ]:
def cellcenter_normal(forme):

 cellCenterCalc = vtk.vtkCellCenters()
 cellCenterCalc.SetInputConnection(forme.GetOutputPort())
 cellCenterCalc.Update()

 pointsCellCenters = cellCenterCalc.GetOutput(0)

 normalsCalc = vtk.vtkPolyDataNormals()
 normalsCalc.SetInputConnection(forme.GetOutputPort())

# Disable normal calculation at cell vertices
 normalsCalc.ComputePointNormalsOff()
# Enable normal calculation at cell centers
 normalsCalc.ComputeCellNormalsOn()
# Disable splitting of sharp edges
 normalsCalc.SplittingOff()
# Disable global flipping of normal orientation
 normalsCalc.FlipNormalsOff()
# Enable automatic determination of correct normal orientation
 normalsCalc.AutoOrientNormalsOn()
# Perform calculation
 normalsCalc.Update()
 
 normalsforme = normalsCalc.GetOutput().GetCellData().GetNormals()
    
 return pointsCellCenters, normalsforme